In [7]:
# birdy client for communication with the server:
from birdy import WPSClient
# import birdy 

# handling files and folders
from os import path, listdir
from urllib import request
import time

# to display external png graphics in notebook: 
from IPython.display import Image
from IPython.core.display import HTML 

In [6]:
# on serverside :
# conda activate finch 
# finch start -c etc/custom.cfg 

fp_server = 'http://localhost:8093/wps'    # flyingpigeon
fp_i = WPSClient(fp_server, progress=True)
fp = WPSClient(fp_server)

finch_server = 'http://localhost:8092/wps'   # finch 
finch_i = WPSClient(url=finch_server, progress=True)
finch = WPSClient(finch_server)

In [15]:
# pr and tas files of bbox over Niger had been prepared
# how to prepare country subsets are explained here:
# https://github.com/nilshempelmann/NAP_Niger/blob/master/subset_polygon.ipynb
#
# Furthermore, biasadjustment had been performed: 
# https://github.com/nilshempelmann/NAP_Niger/blob/master/bias_adjustment.R

# read in the existing bias_adjusted tas files:
tas_path = '/home/nils/nap_niger/data/adjust/'
tas_files = [ tas_path+f for f in listdir(tas_path) if 'tasAdjust' in f]
tas_files.sort()

# read in the existing pr files (!!! not bias adjusted, so far !!!):
pr_path = '/home/nils/nap_niger/data/bbox/'
# lists all files stored in 'path'
pr_files = [ pr_path+f for f in listdir(pr_path) if 'pr_' in f]
pr_files.sort()

print(len(tas_files), len(pr_files) )


8 9


In [71]:
# Quick check of tas datasets ensemble in a plot:
out = fp_i.plot_spaghetti(tas_files)

In [72]:
Image(url= out.get()[0])

In [67]:
# prepare imput parameter 

pr_indices_path = path.join(pr_path,'indices/')
tas_indices_path = path.join(tas_path,'indices/')
# frequencies
freq=['YS','MS']

# precipitation indices
pr_insdices = ['prcptot','rx1day','wetdays','cdd','cwd','sdii','max_n_day_precipitation_amount']
tas_indices = ['tg_mean']

## Claculation of indices:
manipulate the next code box and set the approriate indices with according input files and output information


In [48]:
# bug in finch! Finch calculates only one file even when providing multiple files
# https://github.com/bird-house/finch/issues/72
#
# Solution: loop over single files:

indice_files = []

for i, f  in enumerate(pr_files):
#     # some indices requires pairs. To find corrsponding file:
#     file2 = [t for t in tas_files if path.basename(f).strip('pr_') in t]
#     if len(file2) == 1:
    out = finch_i.max_n_day_precipitation_amount(pr=f, window=5, freq=freq[1]) #  tas=file2[0],
    # wait until finch processed the file:
    while out.getStatus() != 'ProcessSucceeded':
        time.sleep(1)
    filename = path.basename(f).replace('_day_', '_mon_').replace('pr_','max_5_day_precipitation_amount_')
    url = out.get()[0]
    indice_file, _ = request.urlretrieve(url, pr_indices_path+filename)
    indice_files.append(indice_file)
    print('Done for {}'.format(path.basename(indice_file)))
#     else:
#         print('no correspondent file for {} found'.format(path.basename(f)))
        
print('**** Done *****')        
        
# NOTES:
# prcptot requires tas information, should be optional:
# https://github.com/bird-house/finch/issues/71

Done for max_5_day_precipitation_amount_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_mon_19500101-20051230.nc


Done for max_5_day_precipitation_amount_AFR-22_MOHC-HadGEM2-ES_historical_r1i1p1_GERICS-REMO2015_v1_mon_19700101-20051230.nc


Done for max_5_day_precipitation_amount_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_mon_20060101-20981230.nc


Done for max_5_day_precipitation_amount_AFR-22_MOHC-HadGEM2-ES_rcp26_r1i1p1_GERICS-REMO2015_v1_mon_20060101-20991230.nc


Done for max_5_day_precipitation_amount_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_mon_20060101-20991230.nc


Done for max_5_day_precipitation_amount_AFR-22_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_v1_mon_20060101-20701230.nc


Done for max_5_day_precipitation_amount_AFR-22_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_mon_19500101-20051231.nc


Done for max_5_day_precipitation_amount_AFR-22_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_mon_20060101-21001231.nc


Done for max_5_day_precipitation_amount_AFR-22_NCC-NorESM1-M_historical_r1i1p1_CLMcom-KIT-CCLM5-0-15_v1_mon_19500101-20051231.nc
**** Done *****


In [47]:
# Quick check of pr data in a plot:
out = fp_i.plot_spaghetti(indice_files)
while out.getStatus() != 'ProcessSucceeded':
    time.sleep(1)
Image(url= out.get()[0])

In [68]:
# subset polygon for the country  # tas_indices_path
indice_files = [path.join(tas_indices_path,f) for f in listdir(tas_indices_path)]
len(indice_files)

16

In [69]:
# enhancent in flyingpigeon: can't handle multiple files 
# https://github.com/bird-house/flyingpigeon/issues/273

for f in indice_files: 
    out = fp_i.subset_countries(resource=f, region='NER')
    while out.getStatus() != 'ProcessSucceeded':
        time.sleep(1) # wait until the widget is 'Complete'
    url = out.get()[1]
    fn = path.basename(url)
    indice_file, _ = request.urlretrieve(url, path.join(tas_indices_path,fn))

In [70]:
out = fp.plot_map(resource=indice_files[0])
Image(url= out.get()[0])